In [2]:
!pip install langchain openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.2 MB/s eta 0:00:00


In [5]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [12]:
!pip install langchain huggingface_hub

In [39]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [40]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_api_token"

In [41]:
import os
import subprocess
import tempfile
import gradio as gr
from langchain.llms import HuggingFaceHub

In [42]:
huggingface_api_token = "your_huggingface_api_token"

In [51]:
def generate_code(prompt: str) -> str:
    """Generates Python code based on the user prompt using a HuggingFace model."""
    llm = HuggingFaceHub(
        repo_id="bigcode/starcoder",
        model_kwargs={"temperature": 0.1},
    )
    response = llm.invoke(prompt)

    extracted_code = ""
    if "```python" in response:
        extracted_code = response.split("```python\n")[-1].split("```")[0]
    elif "```" in response:
        extracted_code = response.split("```\n")[-1].split("```")[0]
    else:
        extracted_code = response.strip()

    if "def " not in extracted_code:
        return "Error: The model did not generate valid Python code."

    return extracted_code.strip()


In [52]:
def execute_code(code: str) -> str:
    """Executes Python code safely in a temporary file."""
    with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
        temp_file.write(code.encode("utf-8"))
        temp_file_path = temp_file.name

    try:
        result = subprocess.run(["python", temp_file_path], capture_output=True, text=True, timeout=10)
        return result.stdout if result.stdout else result.stderr
    except Exception as e:
        return str(e)

In [53]:
def code_generation_interface(prompt: str):
    """Handles the UI interaction for generating and executing code."""
    code = generate_code(prompt)
    execution_output = execute_code(code)
    return code, execution_output

In [54]:
iface = gr.Interface(
    fn=code_generation_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your coding prompt here..."),
    outputs=[gr.Code(), gr.Textbox(label="Execution Output")],
    title="Code Generator & Executor",
    description="Enter a prompt to generate and execute Python code using Hugging Face models."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e66878abecef86eb4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
